In [ ]:
import os, sys
import pandas as pd
import numpy as np
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

sys.path.append(os.path.abspath('../'))

from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, f1_score
import joblib

RANDOM_STATE = 42
MODEL_DIR = Path('../models')
MODEL_PATH = MODEL_DIR / 'esg_risk_model.joblib'
DATA_PATH = Path('../data/raw/dataset.csv')

pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 50)

print("✅ Libraries loaded successfully")
print(f"📊 Dataset path: {DATA_PATH}")
print(f"💾 Model save path: {MODEL_PATH}")


In [ ]:
df = pd.read_csv(DATA_PATH)
print(f"📂 Dataset loaded: {df.shape[0]} rows × {df.shape[1]} columns")
print(f"\n🔍 First 5 rows:")
df.head()


In [ ]:
df.columns = df.columns.str.strip()

feature_cols = ['Environment Risk Score', 'Social Risk Score', 'Governance Risk Score', 'Controversy Score']
target_col = 'ESG Risk Level'

print(f"🎯 Target column: {target_col}")
print(f"📊 Feature columns: {feature_cols}")

df[target_col] = df[target_col].fillna('Medium')

X = df[feature_cols].copy()
X.columns = ['environment_risk_score', 'social_risk_score', 'governance_risk_score', 'controversy_score']
X = X.fillna(X.median())

label_map = {'Low': 0, 'Medium': 1, 'High': 2}
y = df[target_col].map(label_map).fillna(1).astype(int)

print(f"\n✅ Features shape: {X.shape}")
print(f"✅ Labels shape: {y.shape}")
print(f"\n📊 Class distribution:")
print(y.value_counts().sort_index())
print(f"\n🔢 Feature statistics:")
X.describe()

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=RANDOM_STATE
)

print(f"🔀 Train/Test Split (80/20):")
print(f"   Training set: {X_train.shape[0]} samples")
print(f"   Testing set:  {X_test.shape[0]} samples")
print(f"\n📊 Training class distribution:")
print(y_train.value_counts().sort_index())
print(f"\n📊 Testing class distribution:")
print(y_test.value_counts().sort_index())

In [ ]:
pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('classifier', RandomForestClassifier(
        n_estimators=300,
        max_depth=20,
        min_samples_split=5,
        min_samples_leaf=2,
        random_state=RANDOM_STATE,
        class_weight='balanced',
        n_jobs=-1
    ))
])

print("🏗️ ML Pipeline created:")
print(pipeline)


In [ ]:
print("🚀 Training model...")
pipeline.fit(X_train, y_train)

train_score = pipeline.score(X_train, y_train)
test_score = pipeline.score(X_test, y_test)

print(f"✅ Model trained successfully!")
print(f"📈 Training accuracy: {train_score:.4f}")
print(f"📈 Testing accuracy:  {test_score:.4f}")


In [ ]:
y_pred = pipeline.predict(X_test)

print("📊 Classification Report:")
print("="*60)
print(classification_report(y_test, y_pred, target_names=['Low Risk', 'Medium Risk', 'High Risk']))

print("\n🔲 Confusion Matrix:")
print("="*60)
cm = confusion_matrix(y_test, y_pred)
print(cm)

macro_f1 = f1_score(y_test, y_pred, average='macro')
weighted_f1 = f1_score(y_test, y_pred, average='weighted')

print("\n📈 Overall Metrics:")
print("="*60)
print(f"Macro F1 Score:    {macro_f1:.4f}")
print(f"Weighted F1 Score: {weighted_f1:.4f}")


In [ ]:
MODEL_DIR.mkdir(parents=True, exist_ok=True)
joblib.dump(pipeline, MODEL_PATH)

print("💾 Model saved successfully!")
print(f"📁 Location: {MODEL_PATH}")
print(f"📦 File size: {MODEL_PATH.stat().st_size / 1024:.2f} KB")

loaded_model = joblib.load(MODEL_PATH)
test_prediction = loaded_model.predict(X_test[:5])
print(f"\n✅ Model verification - First 5 predictions: {test_prediction}")
